In [19]:
# Importing necessary libraries
from scipy.sparse import csr_matrix, identity, csc_matrix, lil_matrix #, coo_matrix, dok_matrix, diags
from scipy.sparse.linalg import spsolve #, lsqr, lsmr, 
from scipy.linalg import eig, eigh
import scipy as sp
from numpy import pi, exp, sin, cos
import numpy as np
np.set_printoptions(linewidth=150, precision=15, floatmode='maxprec')
import pandas as pd
# pd.set_option('precision', 15)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [20]:
def finematrix_2d(pfield, Nx, Ny, nx, ny):
    """
                         boundary1
                       ------------
                      |            |
           boundary4  |     K      |  boundary3
                      |            |
                       ------------
                        boundary2
    """
    (hx, hy) = (1/(Nx*nx), 1/(Ny*ny))
    (lx, ly) = (Nx*nx+1, Ny*ny+1)
    (ratiox, ratioy) = (hy/hx, hx/hy)
    
    idx = np.arange(ly*lx).reshape((ly, lx), order='F')
    bdry1 = idx[0, :]      # First row (bdry1)
    bdry2 = idx[-1, :]     # Last  row  (bdry2)
    bdry3 = idx[1:-1, 0]   # bdry3
    bdry4 = idx[1:-1,-1]   # bdry4
    boundary = np.hstack((bdry1, bdry2, bdry3, bdry4))
    
    idx1 = idx[0:-1, 0:-1].flatten(order='F') # array vs vector .reshape((-1, 1), order='F')
    idx2 = (idx1 + ly).flatten(order='F')
    idx3 = (idx1 + ly + 1).flatten(order='F')
    idx4 = (idx1 + 1).flatten(order='F')
    
    lst1 = np.array([2, -2, -1, 1,  -2, 2, 1, -1,  -1, 1, 2, -2,  1, -1, -2, 2]).reshape(-1, 1)
    lst2 = np.array([2, 1, -1, -2,  1, 2, -2, -1,  -1, -2, 2, 1,  -2, -1, 1, 2]).reshape(-1, 1)
    data1= np.kron(lst1*ratiox + lst2*ratioy, pfield.reshape((-1, 1), order='F'))/6
    lst3 = np.array([4, 2, 1, 2,  2, 4, 2, 1,  1, 2, 4, 2,  2, 1, 2, 4]).reshape(-1,1)
    data2= np.kron(lst3, np.ones((pfield.size, 1)))*(hx*hy)/36
    
    #row_ind= np.vstack((idx1,idx2,idx3,idx4, idx1,idx2,idx3,idx4, idx1,idx2,idx3,idx4, idx1,idx2,idx3,idx4))
    #col_ind= np.vstack((idx1,idx1,idx1,idx1, idx2,idx2,idx2,idx2, idx3,idx3,idx3,idx3, idx4,idx4,idx4,idx4))
    row_ind = np.tile(np.vstack((idx1, idx2, idx3, idx4)), (4,1))
    col_ind = np.vstack((np.tile(idx1, (4,1)), np.tile(idx2, (4,1)), np.tile(idx3, (4,1)), np.tile(idx4, (4,1))))
    
    Global_DA = csr_matrix((data1.reshape(-1), (row_ind.reshape(-1), col_ind.reshape(-1)))) #Global Stiffness 
    Global_DA.eliminate_zeros()
    
    Global_M  = csr_matrix((data2.reshape(-1), (row_ind.reshape(-1), col_ind.reshape(-1)))) #Global Mass
    Global_M.eliminate_zeros()
    return Global_DA, Global_M, boundary

In [ ]:
def form_source(f, Nx, Ny, nx, ny):
    (hx, hy) = (1/(Nx*nx), 1/(Ny*ny))
    (lx, ly) = (Nx*nx+1, Ny*ny+1)
    idx = np.arange(ly*lx).reshape((ly, lx), order='F')
    
    idx1 = idx[0:-1, 0:-1].flatten(order='F') # same with .reshape((-1, 1), order='F') #rowwise  'F', colwise 'C'
    idx2 = (idx1 + ly).flatten(order='F')
    idx3 = (idx1 + 1).flatten(order='F')
    idx4 = (idx1 + 1 + ly).flatten(order='F')
    
    data= hx*hy*np.kron(np.ones((4,1)), f.flatten(order='F'))/4
    row_ind = np.vstack((idx1, idx2, idx3, idx4))
    col_ind = np.zeros((4*Nx*nx*Ny*ny, 1),  dtype=int)
    
    F = csr_matrix((data.reshape(-1), (row_ind.reshape(-1), col_ind.reshape(-1))))
    F.eliminate_zeros()
    return F

In [ ]:
def MsFEM_2d_basis(Global_DA, Nx, Ny, nx, ny):
    (lx, ly) = (Nx*nx+1, Ny*ny+1)
    #loc_basis = np.zeros((lx*ly, (Nx+1)*(Ny+1))) # lz = Nz*nz+1 for z in {x, y}
    loc_basis = lil_matrix((lx*ly, (Nx+1)*(Ny+1))) # in some cases lil in others csr is faster
    loc_boundary1 = np.arange(0, (ny+1)*(nx+1), ny+1)                 
    loc_boundary2 = np.arange(ny, (ny+1)*(nx+1), ny+1)          
    loc_boundary3 = np.arange(1, ny)
    loc_boundary4 = np.arange(nx*(ny+1)+1, (ny+1)*(nx+1)-1)
    loc_boundary  = np.hstack((loc_boundary1, loc_boundary2, loc_boundary3, loc_boundary4))
    
    (loc_X, loc_Y) = np.meshgrid(np.linspace(0, 1, nx+1), np.linspace(0, 1, ny+1))
    
    #X.T.flat is same with X.flatten.reshape((-1, 1), order='F') or X.respahe((-1,1), order='F')
    #X.flat gives the flattened matrix in 'C' format, while X.T.flat is in 'F' format
    
    F = np.zeros(((nx+1)*(ny+1), 4))
    F[loc_boundary, 0] = (1-loc_X.T.flat[loc_boundary])*(1-loc_Y.T.flat[loc_boundary])
    F[loc_boundary, 1] = (1-loc_X.T.flat[loc_boundary])*(  loc_Y.T.flat[loc_boundary])
    F[loc_boundary, 2] = (  loc_X.T.flat[loc_boundary])*(1-loc_Y.T.flat[loc_boundary])
    F[loc_boundary, 3] = (  loc_X.T.flat[loc_boundary])*(  loc_Y.T.flat[loc_boundary])
    
    idx = np.arange(lx*ly).reshape(ly, lx, order='F')
    
    for i in np.arange(Nx):
        for j in np.arange(Ny):
            loc_idx = idx[np.ix_(j*ny + np.arange(ny+1), i*nx + np.arange(nx+1))] #np.ix_ for submatrices
            #loc_idx = loc_idx.reshape(-1, order='F')       #loc_idx.T.flat is the same but is an iterator
            loc_A = Global_DA[loc_idx.T.flat, :][:, loc_idx.T.flat].tolil()
            loc_A[loc_boundary, :] = 0
            #loc_A[loc_boundary, :][:, loc_boundary] = identity(2*(ny+nx)) #sparse identity matrix
            loc_A[loc_boundary , loc_boundary] = 1
            loc_basis[np.ix_(loc_idx.T.flat, [i*(Ny+1)+j, i*(Ny+1)+j+1, (i+1)*(Ny+1)+j, (i+1)*(Ny+1)+j+1])] = \
                     spsolve(csr_matrix(loc_A), F, use_umfpack=True) 
            
    return csr_matrix(loc_basis)